In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sp
from matplotlib.ticker import FormatStrFormatter
from math import atan, cos, sin, sqrt

In [79]:
# MEDIDAS INICIALES
codigo = "2190168"
X = sum([int(i) for i in codigo])

# Longitudes de los cables en mm
Longitud_BC_m = 1.5
Longitud_BC_mm = Longitud_BC_m * 1000
Longitud_BD_mm, Longitud_BE_mm = Longitud_BC_mm, Longitud_BC_mm

# Longitud de la barra en mm
Longitud_AB_m = 0.2 * X
Longitud_AB_mm = Longitud_AB_m * 1000

# Diámetro Cables BC, BD, BE; = 1/4"
Diametro_BC_in = 0.25

# Diametro de los cables en mm
Diametro_BC_mm = Diametro_BC_in * 25.4
Diametro_BD_mm, Diametro_BE_mm = Diametro_BC_mm, Diametro_BC_mm

# Pasador = B, C, D, E  diámetro 1/2" 
Diametro_PB_in = 0.5
Diametro_PB_mm = Diametro_PB_in * 25.4
Diametro_PC_mm, Diametro_PD_mm = Diametro_PB_mm, Diametro_PB_mm

# Area de los cables
Area_BC_mm = (np.pi * (Diametro_BC_mm ** 2) )/ 4

# Area de los pasadores
Area_PB_mm = (np.pi * (Diametro_PB_mm ** 2) )/ 4
Area_PC_mm, Area_PD_mm = Area_PB_mm, Area_PB_mm

# Angulo de los cables
Angulo_BE = 50 * (np.pi / 180)
Angulo_BC = 40 *(np.pi / 180)

# Material y sus propiedades 
# Acero ASTM A-36
E = 200  #Gpa
Esfuerzo_fluencia_tension = 250 # Mpa
Esfuerzo_ultimo_tension = 400 #Mpa
Esfuerzo_fluencia_cortante = 145 # Mpa

# Factor de seguridad de la resistencia de fluencia del material
FSF = 1 + (X/100)
# Factor de seguridad de la resistencia última del material
FSU = 1 + (X + 3)/100


In [80]:
# 1.1
# EQUILIBRIO EXTERNO
Ax, Ay, F_BD, F_BC, F_BE, W = sp.symbols("Ax Ay F_BD F_BC F_BE W")

# Sumatoria de fuerzas en Y
Fy = Ay - W*Longitud_AB_mm + F_BD + F_BE*cos(Angulo_BE) + F_BC*cos(Angulo_BC)
sumatoria_Fy = sp.Eq(Fy, 0)
sumatoria_Fy


Eq(Ay + 0.766044443118978*F_BC + F_BD + 0.642787609686539*F_BE - 5400.0*W, 0)

In [81]:
# Sumatoria de fuerzas en X
Fx = Ax + F_BE*sin(Angulo_BE) - F_BC*sin(Angulo_BC)
sumatoria_Fx = sp.Eq(Fx, 0)
sumatoria_Fx


Eq(Ax - 0.642787609686539*F_BC + 0.766044443118978*F_BE, 0)

In [82]:
# Sumatoria de momentos en el punto A
Ma = -W*(Longitud_AB_mm**2)/2 + Longitud_AB_mm*F_BD + Longitud_AB_mm*F_BE*cos(Angulo_BE) + Longitud_AB_mm*F_BC*cos(Angulo_BC)
sumatoria_Ma = sp.Eq(Ma, 0)
sumatoria_Ma

Eq(4136.63999284248*F_BC + 5400.0*F_BD + 3471.05309230731*F_BE - 14580000.0*W, 0)

In [83]:
# CABLES
# Relacion BD y BC
F_BD_BC = sp.Eq(F_BC - F_BD*cos(Angulo_BC), 0)
relacion_F_BD_BC = F_BD*cos(Angulo_BC)
F_BD_BC

Eq(F_BC - 0.766044443118978*F_BD, 0)

In [84]:
# Relacion BD y BE
F_BD_BE = sp.Eq(F_BE - F_BD*cos(Angulo_BE), 0)
relacion_F_BD_BE = F_BD*cos(Angulo_BE)
F_BD_BE

Eq(-0.642787609686539*F_BD + F_BE, 0)

In [85]:
# Fuerzas BC y BE en terminos de BD
momentos = sp.Eq(Ma.subs([(F_BC, relacion_F_BD_BC), (F_BE, relacion_F_BD_BE)]), 0)
momentos


Eq(10800.0*F_BD - 14580000.0*W, 0)

In [86]:
# Fuerza BD en terminos de W
F_BD_W = sp.solve(momentos)[0][F_BD]
F_BD_W


1350.0*W

In [87]:
# Carga máxima para el cable BD
esfuerzo_F_BD = (F_BD_W / Area_BC_mm) - (Esfuerzo_ultimo_tension / FSU)
equivalencia_F_BD_W = sp.Eq(esfuerzo_F_BD, 0)
equivalencia_F_BD_W

Eq(42.6281452140237*W - 307.692307692308, 0)

In [88]:
carga_maxima_F_BD_W = sp.solve(equivalencia_F_BD_W, W)
print("La carga máxima distribuida que puede soportar el cable BD es de " + str(carga_maxima_F_BD_W[0]) + " kN/m")


La carga máxima distribuida que puede soportar el cable BD es de 7.21805525779912 kN/m


In [89]:
# Carga máxima para el cable BC
esfuerzo_F_BC = (relacion_F_BD_BC.subs(F_BD, F_BD_W) / Area_BC_mm) - (Esfuerzo_ultimo_tension / FSU)
equivalencia_F_BC_BD = sp.Eq(esfuerzo_F_BC, 0)
equivalencia_F_BC_BD


Eq(32.6550537616717*W - 307.692307692308, 0)

In [90]:
carga_maxima_F_BC_W = sp.solve(equivalencia_F_BC_BD, W)
print("La carga máxima distribuida que puede soportar el cable BD es de " + str(carga_maxima_F_BC_W[0]) + " kN/m")


La carga máxima distribuida que puede soportar el cable BD es de 9.42250194833415 kN/m


In [91]:
# Carga máxima para el cable BE
esfuerzo_F_BE = (relacion_F_BD_BE.subs(F_BD, F_BD_W) / Area_BC_mm) - (Esfuerzo_ultimo_tension / FSU)
equivalencia_F_BE_BD = sp.Eq(esfuerzo_F_BE, 0)
equivalencia_F_BE_BD


Eq(27.400843567493*W - 307.692307692308, 0)

In [92]:
carga_maxima_F_BE_W = sp.solve(equivalencia_F_BE_BD, W)
print("La carga máxima distribuida que puede soportar el cable BD es de " + str(carga_maxima_F_BE_W[0]) + " kN/m")


La carga máxima distribuida que puede soportar el cable BD es de 11.2293005481532 kN/m


In [93]:
# PASADOR
# Pasador A
R_Ay = Fy.subs([(F_BC, relacion_F_BD_BC), (F_BE, relacion_F_BD_BE), (F_BD, F_BD_W)])
equivalencia_R_Ay = sp.solve(sp.Eq(R_Ay, 0))
equivalencia_R_Ay = equivalencia_R_Ay[0][Ay]
equivalencia_R_Ay


2700.0*W

In [94]:
R_Ax = Fx.subs([(F_BC, relacion_F_BD_BC), (F_BE, relacion_F_BD_BE), (F_BD, F_BD_W)])
equivalencia_R_Ax = sp.solve(sp.Eq(R_Ax, 0))
equivalencia_R_Ax = equivalencia_R_Ax[0][Ax]
equivalencia_R_Ax


-1.49880108324396e-13*W

In [95]:
# Resultante A en terminos de W
R_A = sp.sqrt(equivalencia_R_Ay**2 + equivalencia_R_Ax**2)
R_A

2700.0*sqrt(W**2)

In [96]:
# Carga máxima para el pasador A
esfuerzo_R_A = (R_A / Area_PB_mm) - (Esfuerzo_fluencia_cortante / FSF)
equivalencia_R_A = sp.Eq(esfuerzo_R_A, 0)
equivalencia_R_A

Eq(21.3140726070118*sqrt(W**2) - 114.173228346457, 0)

In [97]:
W1 = sp.sqrt(W**2)
carga_maxima_R_A = sp.solve(equivalencia_R_A, W1)
print("La carga máxima distribuida que puede soportar el pasador A es de " + str(carga_maxima_R_A[0]) + " kN/m")

La carga máxima distribuida que puede soportar el pasador A es de 5.35670636257929 kN/m


In [98]:
# Pasador B
R_By = relacion_F_BD_BC.subs(F_BD, F_BD_W)*cos(Angulo_BC) + relacion_F_BD_BE.subs(F_BD, F_BD_W)*cos(Angulo_BE) + F_BD_W
R_Bx = relacion_F_BD_BE.subs(F_BD, F_BD_W)*sin(Angulo_BE) - relacion_F_BD_BC.subs(F_BD, F_BD_W)*sin(Angulo_BC) 
R_B = sp.sqrt(R_Bx**2 + R_By**2)
R_B

2700.0*sqrt(W**2)

In [99]:
# Carga máxima para el pasador B
esfuerzo_R_B = (R_B / Area_PB_mm) - (Esfuerzo_fluencia_cortante / FSF)
equivalencia_R_B = sp.Eq(esfuerzo_R_B, 0)
equivalencia_R_B

Eq(21.3140726070118*sqrt(W**2) - 114.173228346457, 0)

In [100]:
W1 = sp.sqrt(W**2)
carga_maxima_R_B = sp.solve(equivalencia_R_B, W1)
print("La carga máxima distribuida que puede soportar el pasador B es de " + str(carga_maxima_R_B[0]) + " kN/m")

La carga máxima distribuida que puede soportar el pasador B es de 5.35670636257929 kN/m


In [101]:
# Carga máxima para el pasador BC
esfuerzo_P_F_BC = (relacion_F_BD_BC.subs(F_BD, F_BD_W) / Area_PB_mm) - (Esfuerzo_fluencia_cortante / FSF)
equivalencia_P_F_BC = sp.Eq(esfuerzo_P_F_BC, 0)
equivalencia_P_F_BC

Eq(8.16376344041793*W - 114.173228346457, 0)

In [102]:
carga_maxima_P_BC = sp.solve(equivalencia_P_F_BC, W)
print("La carga máxima distribuida que puede soportar el pasador del cable BC es de " + str(carga_maxima_P_BC[0]) + " kN/m")

La carga máxima distribuida que puede soportar el pasador del cable BC es de 13.9853670650472 kN/m


In [103]:
# Carga máxima para el pasador BD
esfuerzo_P_F_BD = (F_BD_W / Area_PB_mm) - (Esfuerzo_fluencia_cortante / FSF)
equivalencia_P_F_BD = sp.Eq(esfuerzo_P_F_BD, 0)
equivalencia_P_F_BD

Eq(10.6570363035059*W - 114.173228346457, 0)

In [104]:
carga_maxima_P_BD = sp.solve(equivalencia_P_F_BD, W)
print("La carga máxima distribuida que puede soportar el pasador del cable BD es de " + str(carga_maxima_P_BD[0]) + " kN/m")

La carga máxima distribuida que puede soportar el pasador del cable BD es de 10.7134127251586 kN/m


In [105]:
# Carga máxima para el pasador BE
esfuerzo_P_F_BE = (relacion_F_BD_BE.subs(F_BD, F_BD_W) / Area_PB_mm) - (Esfuerzo_fluencia_cortante / FSF)
equivalencia_P_F_BE = sp.Eq(esfuerzo_P_F_BE, 0)
equivalencia_P_F_BE

Eq(6.85021089187324*W - 114.173228346457, 0)

In [106]:
carga_maxima_P_BE = sp.solve(equivalencia_P_F_BE, W)
print("La carga máxima distribuida que puede soportar el pasador del cable BE es de " + str(carga_maxima_P_BE[0]) + " kN/m")


La carga máxima distribuida que puede soportar el pasador del cable BE es de 16.6671114435187 kN/m


In [107]:
W_max = min([carga_maxima_F_BD_W, carga_maxima_F_BC_W, carga_maxima_F_BE_W, carga_maxima_P_BC, carga_maxima_P_BD, carga_maxima_P_BE, carga_maxima_R_B, carga_maxima_R_A])
W_maxima = W_max[0]
print("La carga máxima distribuida para que ningún elemento falle es de " + str(W_maxima) + " kN/m")

La carga máxima distribuida para que ningún elemento falle es de 5.35670636257929 kN/m


In [108]:
# 1.2
# FUERZAS
# Cable BD
carga_F_BD = F_BD_W.subs(W, W_maxima)/1000
print("La fuerza del cable BD es de " + str(carga_F_BD) + " kN")

La fuerza del cable BD es de 7.23155358948204 kN


In [109]:
# Cable BC
carga_F_BC = relacion_F_BD_BC.subs(F_BD, F_BD_W)
carga_F_BC = carga_F_BC.subs(W, W_maxima)/1000
print("La fuerza del cable BC es de " + str(carga_F_BC) + " kN")

La fuerza del cable BC es de 5.53969144233981 kN


In [110]:
# Cable BE
carga_F_BE = relacion_F_BD_BE.subs(F_BD, F_BD_W)
carga_F_BE = carga_F_BE.subs(W, W_maxima)/1000
print("La fuerza del cable BC es de " + str(carga_F_BE) + " kN")

La fuerza del cable BC es de 4.64835304610327 kN


In [111]:
# DEFORMACIONES
# Cable BD
deformacion_F_BD = (carga_F_BD * Longitud_BC_mm) / (Area_BC_mm * E)
print("La deformación del cable BD es de " + str(deformacion_F_BD) + " mm")

La deformación del cable BD es de 1.71259842519686 mm


In [112]:
# Cable BC
deformacion_F_BC = (carga_F_BC * Longitud_BC_mm) / (Area_BC_mm * E)
print("La deformación del cable BC es de " + str(deformacion_F_BC) + " mm")

La deformación del cable BC es de 1.31192650691637 mm


In [113]:
# Cable BE
deformacion_F_BE = (carga_F_BE * Longitud_BC_mm) / (Area_BC_mm * E)
print("La deformación del cable BE es de " + str(deformacion_F_BE) + " mm")

La deformación del cable BE es de 1.10083704808522 mm


In [114]:
# REACCIONES
# Apoyo A
reaccion_Ax = Fx.subs([(F_BC, carga_F_BC), (F_BE, carga_F_BE)])
reaccion_Ax = sp.solve(sp.Eq(reaccion_Ax,0))[0]
print("La reaccion Ax es de " + str(reaccion_Ax) + " kN")

La reaccion Ax es de -4.44089209850063e-16 kN


In [115]:
if reaccion_Ax == 0:
    reaccion_Ay = R_A.subs([(equivalencia_R_Ax, reaccion_Ax), (W, W_maxima)])/1000
print("La reaccion Ay es de " + str(reaccion_Ay) + " kN")

La reaccion Ay es de 14.6945168938274 kN
